In [1]:
import sys
import os

import requests
import json

import pandas as pd

from web3 import Web3

# SCRIPT_DIR = os.path.dirname(os.path.abspath('__file__'))
PROJECT_DIR = os.path.dirname(os.path.abspath(''))
sys.path.append(os.path.dirname(PROJECT_DIR))

from pyDpi.util import setup
from pyDpi.util.libs import invoke_contract

seleciona a menor colecao

In [2]:
w3 = setup.load_blockchain_driver()
deployed_contracts = setup.load_deployed_smart_contracts(w3)
dpid_db = deployed_contracts['PidDB.sol']
epid_db = deployed_contracts['ExternalPidDB.sol']
auth_db = deployed_contracts['AuthoritiesDB.sol']

dpid_service = deployed_contracts['PIDService.sol']
epid_service = deployed_contracts['ExternalPIDService.sol']
sets_service = deployed_contracts['SearchTermService.sol']
auth_service = deployed_contracts['AuthoritiesService.sol']


chain_id,min_gas_price,pk = setup.get_exec_parameters()
account = w3.eth.account.privateKeyToAccount(pk)

/home/edilson/Workspace/D-pi/config.ini


## Configuração do ambiente

variaveis

In [3]:
nam = '8003'
ror_id = '3wf3'
addr = '0xf17f52151EbEF6C7334FAD080c5704D77216b732'
dshoulder_prefix = 'fkwf'

### criar dnma

- criar metodo para alterar

In [4]:
recipt_tx = invoke_contract(w3,account,chain_id, auth_service , 'create_dnam', ror_id , dshoulder_prefix, addr)
auth_id = recipt_tx['logs'][0]['topics'][1]
auth_id

HexBytes('0x47be0ba38f0ab57d6469dedd62c007b4605d000efe0ceeade9696543076c59bc')

In [5]:
auth_db.caller.count_dnma()
Web3.toHex( auth_db.caller.get_dnma_by_index(0) ) # converte o indice 0 
msg = "DecentralizedNameMappingAuthority :: {} [ ror={} , shoulder_prefix={} ]".format(auth_db.caller.get_dnma(auth_id)[-1],
                                                                                        auth_db.caller.get_dnma(auth_id)[1],
                                                                                        auth_db.caller.get_dnma(auth_id)[1])

msg

'DecentralizedNameMappingAuthority :: 0xf17f52151EbEF6C7334FAD080c5704D77216b732 [ ror=3wf3 , shoulder_prefix=3wf3 ]'

configura o noid_provider da autoridade
 - nam ( ibict 8003)
 - addr da autho
 - noid_len : tamanho do noid
 - type (1 - dnma, 2 - sma)

In [6]:
noid_len = 5

r_tx = invoke_contract(w3,account,chain_id, auth_service , 'configure_noid_provider_dnma', nam, auth_id , noid_len , 1)
noid_addr = r_tx['logs'][0]['topics'][1][12:]
noid_addr # endereco do contrato

HexBytes('0x2b1628dda8dfffaf219cb00a1009b5ca66489702')

### pid service

#### create a noid

In [7]:
r_tx = invoke_contract(w3,account,chain_id, dpid_service , 'assingID', addr)
dark_id = r_tx['logs'][0]['topics'][1]
dark_id

HexBytes('0x4d43c455dad9fba4fc813875117506782acd79501649077412a359a0f16805b7')

In [8]:
# Web3.toHex( dpid_db.caller.get(dark_id)[0] )
dpid_db.caller.get(dark_id)[1]

'8003/fkwff300001v'

##### create a metada

search term

In [9]:
r_tx = invoke_contract(w3,account,chain_id, dpid_service, 'addSearchTerm',dark_id ,'DARK')
r_tx = invoke_contract(w3,account,chain_id, dpid_service, 'addSearchTerm',dark_id, 'ARK')

external pid

In [10]:
r_tx = invoke_contract(w3,account,chain_id, dpid_service, 'addExternalPid',dark_id, 'DOI' , '10.1016/j.is.2021.101826') 

external link

In [11]:
r_tx = invoke_contract(w3,account,chain_id, dpid_service, 'add_externalLinks',dark_id, 'https://doi.org/10.1016/j.is.2021.101826') 

payload

In [12]:
payload_data = { 'title' : 'Blockchain-based Privacy-Preserving Record Linkage: enhancing data privacy in an untrusted environment.' ,
                 'author' : 'Thiago Nóbrega',
                 'type' : 'Article'
            }


r_tx = invoke_contract(w3,account,chain_id, dpid_service, 'set_payload',dark_id, str(payload_data)) 